In [1]:
import tkinter as tk
from tkinter import Canvas
import geojson
import json
import random
import requests

In [2]:
def convert_coordinates_to_pixels(latitude, longitude, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon):
    # Convert latitude and longitude to pixel values
    x = (longitude - min_lon) / (max_lon - min_lon) * canvas_width
    y = canvas_height - (latitude - min_lat) / (max_lat - min_lat) * canvas_height
    return int(x), int(y)

In [3]:
def draw_geojson_feature(feature, canvas, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon, color):
    # Get the geometry of the feature
    geometry = feature.get('geometry')

    if geometry is not None:
        # Convert the coordinates to pixel values
        coordinates = geometry['coordinates']
        geometry_type = geometry['type']

        if geometry_type == 'Point':
            lon, lat = coordinates
            x, y = convert_coordinates_to_pixels(lat, lon, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon)
            canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill=color)

        elif geometry_type == 'LineString':
            coords_pixel = []
            for lon, lat in coordinates:
                x, y = convert_coordinates_to_pixels(lat, lon, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon)
                coords_pixel.append((x, y))
            canvas.create_line(coords_pixel, fill=color, width=2)

        elif geometry_type == 'Polygon':
            for ring_coords in coordinates:
                coords_pixel = []
                for lon, lat in ring_coords:
                    x, y = convert_coordinates_to_pixels(lat, lon, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon)
                    coords_pixel.append((x, y))
                canvas.create_polygon(coords_pixel, outline=color, fill=color, width=2)

        elif geometry_type == 'MultiPoint':
            for point_coords in coordinates:
                lon, lat = point_coords
                x, y = convert_coordinates_to_pixels(lat, lon, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon)
                canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill=color)

        elif geometry_type == 'MultiLineString':
            for line_coords in coordinates:
                coords_pixel = []
                for lon, lat in line_coords:
                    x, y = convert_coordinates_to_pixels(lat, lon, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon)
                    coords_pixel.append((x, y))
                canvas.create_line(coords_pixel, fill=color, width=2)

        elif geometry_type == 'MultiPolygon':
            for poly_coords in coordinates:
                for ring_coords in poly_coords:
                    coords_pixel = []
                    for lon, lat in ring_coords:
                        x, y = convert_coordinates_to_pixels(lat, lon, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon)
                        coords_pixel.append((x, y))
                    canvas.create_polygon(coords_pixel, outline=color, fill=color, width=2)

        elif geometry_type == 'GeometryCollection':
            # Handle any geometries within the collection
            for sub_geometry in geometry['geometries']:
                draw_geojson_feature(sub_geometry, canvas, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon, color)
        # Add handling for other geometry types if needed


In [4]:
def get_screen_size(root):
    # Get the width and height of the user's screen
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    return screen_width, screen_height

In [5]:
def find_min_max_coordinates(geojson_file):
    # Read GeoJSON file with the correct encoding
    with open(geojson_file, 'r', encoding='utf-8') as f:
        data = geojson.load(f)

    # Initialize variables to store min and max coordinates
    min_lat, max_lat = float('inf'), float('-inf')
    min_lon, max_lon = float('inf'), float('-inf')

    # Helper function to update min and max coordinates
    def update_min_max(lat, lon):
        nonlocal min_lat, max_lat, min_lon, max_lon
        min_lat = min(min_lat, lat)
        max_lat = max(max_lat, lat)
        min_lon = min(min_lon, lon)
        max_lon = max(max_lon, lon)

    # Iterate through features to find min and max coordinates
    for feature in data['features']:
        geometry = feature.get('geometry')
        if geometry is not None:
            geometry_type = geometry['type']
            coordinates = geometry['coordinates']

            if geometry_type == 'Point':
                lon, lat = coordinates
                update_min_max(lat, lon)

            elif geometry_type == 'LineString':
                for lon, lat in coordinates:
                    update_min_max(lat, lon)

            elif geometry_type == 'Polygon':
                for ring in coordinates:
                    for lon, lat in ring:
                        update_min_max(lat, lon)

            elif geometry_type == 'MultiPoint':
                for point_coords in coordinates:
                    lon, lat = point_coords
                    update_min_max(lat, lon)

            elif geometry_type == 'MultiLineString':
                for line_coords in coordinates:
                    for lon, lat in line_coords:
                        update_min_max(lat, lon)

            elif geometry_type == 'MultiPolygon':
                for poly_coords in coordinates:
                    for ring in poly_coords:
                        for lon, lat in ring:
                            update_min_max(lat, lon)

            elif geometry_type == 'GeometryCollection':
                # Handle any geometries within the collection
                for sub_geometry in geometry['geometries']:
                    if sub_geometry['type'] == 'Point':
                        lon, lat = sub_geometry['coordinates']
                        update_min_max(lat, lon)
                    elif sub_geometry['type'] == 'LineString':
                        for lon, lat in sub_geometry['coordinates']:
                            update_min_max(lat, lon)
                    elif sub_geometry['type'] == 'Polygon':
                        for ring in sub_geometry['coordinates']:
                            for lon, lat in ring:
                                update_min_max(lat, lon)
                    # Add handling for other geometry types if needed

    return min_lat, max_lat, min_lon, max_lon


In [6]:
def is_road_feature(properties):
    # Check if the feature is a road or street based on its properties
    return 'highway' in properties

In [10]:
def get_coordinates_from_address(address):
    url = f'https://nominatim.openstreetmap.org/search?q={address}&format=json'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = float(data[0]['lat'])
            lon = float(data[0]['lon'])
            return lat, lon
    return None, None


def fetch_features_from_overpass_api(min_lat, max_lat, min_lon, max_lon):
    overpass_url = 'https://overpass-api.de/api/interpreter'
    overpass_query = f"""
    [out:json];
    (
        node({min_lat},{min_lon},{max_lat},{max_lon});
        way({min_lat},{min_lon},{max_lat},{max_lon});
        relation({min_lat},{min_lon},{max_lat},{max_lon});
    );
    out;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    if response.status_code == 200:
        data = response.json()
        return data
    return None

In [8]:
with open(r'C:\Users\gitan\ML_Projects\Mapper\queens_park.osm.geojson\queens_park.osm.geojson', 'r', encoding='utf-8') as f:
        data = geojson.load(f)

In [9]:
# Find the minimum and maximum latitude and longitude in the GeoJSON file
geojson_file = r'C:\Users\gitan\ML_Projects\Mapper\queens_park.osm.geojson\queens_park.osm.geojson'
min_lat, max_lat, min_lon, max_lon = find_min_max_coordinates(geojson_file)


In [11]:
# Create Tkinter window
root = tk.Tk()
root.title('GeoJSON Feature Viewer')

canvas_bg_color = '#333333'  # Dark gray color
# root.configure(bg=canvas_bg_color)

# Get the width and height of the canvas
screen_width, screen_height = get_screen_size(root)

first_feature = data['features'][0] if 'features' in data else None

if first_feature is not None:
    # Create a canvas
    canvas_width, canvas_height = screen_width, screen_height

    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg=canvas_bg_color)
    canvas.pack()


    highway_colors = {
        'motorway': '#ffc0cb',      # Pastel pink
        'trunk': '#add8e6',         # Pastel blue
        'primary': '#98fb98',       # Pastel green
        'secondary': '#ffdead',     # Pastel yellow
        'tertiary': '#ffb6c1',      # Pastel rose
        'residential': '#dda0dd'    # Pastel purple
    }

    # Function to generate random pastel colors for remaining highway types
    def generate_random_pastel_color():
        return '#%02x%02x%02x' % (random.randint(64, 224), random.randint(64, 224), random.randint(64, 224))



    # Draw the first feature on the canvas
    valid_highway_types = {'motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'residential, track, '}
    road_color = '#e4e4e4'
    building_color = '#FFD700'
    park_color = '#32CD32'


    zoom_factor = 1.0
    offset_x, offset_y = 0, 0



    for feature in data['features']:
        geometry_type = feature.get('geometry', {}).get('type', '')
        properties = feature.get('properties', {})
        highway_type = properties.get('highway')

        if highway_type in valid_highway_types:
            road_color = highway_colors.get(highway_type, generate_random_pastel_color())
            draw_geojson_feature(feature, canvas, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon, road_color)
        
        building_type = properties.get('building')
        if building_type is not None:
            draw_geojson_feature(feature, canvas, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon, building_color)

        leisure_type = properties.get('leisure')
        if leisure_type == 'park':
            draw_geojson_feature(feature, canvas, canvas_width, canvas_height, min_lat, max_lat, min_lon, max_lon, park_color)
            
    # Function to handle mouse wheel event for zooming
    def on_mousewheel(event):
        global zoom_factor
        if event.delta > 0:
            zoom_factor *= 1.1
        else:
            zoom_factor *= 0.9
        canvas.scale('all', event.x, event.y, zoom_factor, zoom_factor)

    # Function to handle mouse drag event for panning
    def on_mouse_drag(event):
        global offset_x, offset_y
        canvas.move('all', event.x - offset_x, event.y - offset_y)
        offset_x, offset_y = event.x, event.y

    # Bind mouse wheel and drag events to canvas
    canvas.bind('<MouseWheel>', on_mousewheel)
    canvas.bind('<B1-Motion>', on_mouse_drag)


    # Run the Tkinter main loop
    root.mainloop()